<a href="https://colab.research.google.com/github/EmiMontalto/Analisis-V1/blob/main/%22FinBERT_para_Decisiones_de_Trading_Compra_y_Venta_Predictiva_a_partir_de_Datos_Hist%C3%B3ricos_de_Criptomonedas%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por supuesto, aquí está un resumen de los acuerdos y características para el desarrollo del modelo:

---

**Objetivo del Proyecto**:
Desarrollar un modelo de inteligencia artificial basado en FinBERT para predecir, con un día de anticipación, si se debe comprar o vender una acción. La decisión es binaria: compra o venta. El modelo se entrenará utilizando datos históricos de precios de criptos y se validará mediante validación cruzada en el tiempo. La fucnion def detect_and_plot_peaks(data), marcara la correcta divicion de la informacion y la columna que da por resultado dara la informacion contra la cual contrastar la info.

**Características del Modelo**:

Para avanzar con el uso de FinBERT en tu proyecto de predicción de señales de compra o venta para acciones de criptomonedas, aquí está un esquema de alto nivel del proceso que podríamos seguir:

### Paso 1: Preparación de Datos
1. **Recopilación de Datos**: Asegúrate de que los datos históricos de las criptomonedas están completos y limpios.
2. **Generación de Características**: Calcula los indicadores técnicos necesarios y estandariza cualquier dato numérico que se vaya a utilizar.
3. **Conversión a Texto**: Transforma las características numéricas en descripciones textuales que FinBERT pueda entender, manteniendo un contexto financiero relevante.
4. **Etiquetado de Datos**: Utiliza las funciones que detectan picos alcistas y bajistas para etiquetar los datos con señales de 'compra' o 'venta'.
5. **División de Datos**: Prepara conjuntos de datos de entrenamiento, validación y prueba, respetando la estructura de series temporales.

### Paso 2: Ajuste Fino del Modelo
1. **Carga de FinBERT**: Carga el modelo FinBERT preentrenado.
2. **Adaptación del Tokenizador**: Adapta el tokenizador de FinBERT para manejar cualquier formato especial que hayas diseñado en la preparación de datos.
3. **Procesamiento de Entrada**: Convierte tus descripciones textuales en el formato de entrada que requiere FinBERT (tokens, máscaras de atención, etc.).
4. **Configuración de Entrenamiento**: Establece la función de pérdida (por ejemplo, entropía cruzada para clasificación), el optimizador (Adam suele ser una buena elección) y cualquier otro hiperparámetro.
5. **Entrenamiento**: Entrena FinBERT en tu conjunto de datos, utilizando técnicas como el entrenamiento por lotes y la validación cruzada en el tiempo.
6. **Validación y Ajuste**: Evalúa el modelo en el conjunto de datos de validación y realiza los ajustes necesarios en los hiperparámetros.

### Paso 3: Evaluación del Modelo
1. **Evaluación de Rendimiento**: Usa el conjunto de datos de prueba para evaluar el rendimiento del modelo utilizando métricas como precisión, recall y F1-score.
2. **Interpretación de Resultados**: Analiza los resultados para entender cómo el modelo está tomando decisiones y qué tan bien se correlacionan con las señales reales de trading.

### Paso 4: Despliegue del Modelo
1. **Selección del Modelo**: Elige el mejor modelo basado en las métricas de evaluación.
2. **Guardado del Modelo**: Guarda el modelo y cualquier procesamiento asociado de datos para que pueda ser utilizado consistentemente.
3. **Integración en la Estrategia de Trading**: Integra el modelo en un sistema de trading automatizado o un flujo de trabajo donde las señales pueden ser utilizadas para tomar decisiones de trading.

### Paso 5: Monitoreo y Mantenimiento
1. **Monitoreo Continuo**: Monitorea el rendimiento del modelo en el tiempo para asegurar que sigue siendo válido a medida que se recopilan nuevos datos.
2. **Reentrenamiento Periódico**: Planifica reentrenar el modelo periódicamente o cuando surjan cambios significativos en el mercado.

Este esquema es solo un punto de partida. Cada uno de estos pasos puede requerir un trabajo detallado y ajustes específicos para adaptarse a las particularidades de los datos y los requisitos de tu proyecto. ¿Hay algún paso específico en el que quieras profundizar o iniciar el proceso?

```


In [1]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=02b1cd50ce74affeca213063580514bb5825ba4d4ebae70f3a243d31bd531f98
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00


In [3]:
pip install accelerate -U


In [4]:
pip install transformers

In [5]:
from google.colab import  drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [7]:


def unify_detect_label_peaks(data):
    # Detectar picos alcistas y bajistas
    bullish_peaks, _ = find_peaks(data['Close'], distance=10, prominence=1)
    bearish_peaks, _ = find_peaks(-data['Close'], distance=10, prominence=1)

    # Inicializar la señal con venta hasta que se encuentre el primer pico de compra
    data['Numeric_Signal'] = -1  # Todos los puntos inicialmente a venta

    # Ordenar y combinar los picos
    all_peaks = np.sort(np.concatenate((bullish_peaks, bearish_peaks)))

    # Verificar si se encontraron picos
    if all_peaks.size > 0:
        # Alternar señales entre los picos
        signal = 1
        for i in range(len(all_peaks)-1):
            # Asignar señal de compra o venta entre los picos usando .loc para evitar SettingWithCopyWarning
            data.loc[all_peaks[i]:all_peaks[i+1], 'Numeric_Signal'] = signal
            signal *= -1  # Cambiar señal para el próximo segmento

        # Asegurarse de que el último segmento después del último pico también tenga señal
        if all_peaks[-1] < len(data) - 1:
            data.loc[all_peaks[-1]:, 'Numeric_Signal'] = signal

    # Mapear señales numéricas a señales de trading
    data['Trade_Signal'] = data['Numeric_Signal'].map({1: 'compra', -1: 'venta'})

    return data


In [8]:
import pandas as pd
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# Paso 1: Preprocesamiento de Datos

def load_and_preprocess_data(crypto_list, data_folder):
    all_data = pd.DataFrame()  # DataFrame para concatenar todos los datos

    for crypto in crypto_list:
        file_path = os.path.join(data_folder, f"{crypto}_daily_normalized.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            # Aquí puedes aplicar cualquier preprocesamiento adicional necesario

            # Luego etiquetas los datos con la función unify_detect_label_peaks
            # Asumimos que esta función ya está definida en el contexto
            df = unify_detect_label_peaks(df)  # No necesitamos la trama aquí

            # Concatenamos los datos de esta cripto con el DataFrame global
            all_data = pd.concat([all_data, df], ignore_index=True)
        else:
            print(f"No se encontró el archivo para {crypto}")

    return all_data

# Asumiendo que la función unify_detect_label_peaks ya está definida y disponible en el contexto






In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
# Paso 2: Carga y Ajuste Fino de FinBERT


def fine_tune_finbert(preprocessed_data):
    # Convertir las características numéricas en texto
    preprocessed_data['text_data'] = preprocessed_data.apply(
        lambda x: f"ZScore {x['ZScore']} Close_21SMA_Ratio {x['Close_21SMA_Ratio']} OHLC_21SMA_Ratio {x['OHLC_21SMA_Ratio']} "
                  f"rsi_2 {x['rsi_2']} RSI2_RSI14_Ratio {x['RSI2_RSI14_Ratio']} ADX {x['ADX']} ADX_pos {x['ADX_pos']} "
                  f"ADX_neg {x['ADX_neg']} BB_Upper_Ratio {x['BB_Upper_Ratio']} BB_Lower_Ratio {x['BB_Lower_Ratio']} Diff {x['Diff']}",
        axis=1
    )
    # Suponiendo que preprocessed_data['text_data'] ya está correctamente formateado como una lista de strings
    text_data = list(preprocessed_data['text_data'])

    # Cargar el tokenizador de FinBERT
    tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")


    # Tokenizar los datos de texto
    tokens = tokenizer(
    text_data,
    padding=True,
    truncation=True,
    return_tensors="pt"
    )
    # Asegúrate de que 'tokens' es un diccionario
    if not isinstance(tokens, dict):
        raise ValueError("El tokenizador no está devolviendo un diccionario como se esperaba.")

    # Convertir las etiquetas en un formato adecuado para la clasificación binaria
    labels = preprocessed_data['Numeric_Signal'].map({1: 1, -1: 0}).values

    # Dividir los datos en conjuntos de entrenamiento y validación
    train_data, validation_data, train_labels, validation_labels = train_test_split(
        tokens['input_ids'], labels, test_size=0.1, random_state=42
    )

    # Asegurarse de que todos los datos son tensores
    train_data = torch.tensor(train_data)
    validation_data = torch.tensor(validation_data)
    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(validation_labels)

    # Convertir los conjuntos de datos a formato de PyTorch
    train_dataset = TensorDataset(train_data, train_labels)
    valid_dataset = TensorDataset(validation_data, validation_labels)

    # Cargar el modelo FinBERT preentrenado
    model = AutoModelForSequenceClassification.from_pretrained(
        "yiyanghkust/finbert-tone",
        num_labels=2,  # Especificamos que solo tenemos 2 etiquetas para la clasificación
        ignore_mismatched_sizes=True  # Agregamos este argumento para ignorar el desajuste de tamaño
    )
    # Argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=False,  # Cargar el mejor modelo al final del entrenamiento
        evaluation_strategy="epoch",  # Evaluación al final de cada época
    )

    # Instanciar Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer
    )

    # Entrenar el modelo
    trainer.train()

    return model, tokenizer, trainer


In [10]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# Paso 3: Entrenamiento y Evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def train_and_evaluate(model, tokenizer, train_data, validation_data):
    # Argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        evaluation_strategy="epoch",  # Evaluación al final de cada época
        save_strategy="epoch",        # Guardar al final de cada época para que coincida con evaluation_strategy
        load_best_model_at_end=True   # Cargar el mejor modelo al final del entrenamiento
    )

    # Instanciar Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=validation_data,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    # Entrenar el modelo
    train_result = trainer.train()
    trainer.save_model()  # Guarda el mejor modelo

    # Evaluación
    metrics = trainer.evaluate()

    # Devuelve resultados
    return train_result, metrics


In [11]:
def save_model(model, tokenizer, save_directory):
    # Crear el directorio si no existe
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Guardar el modelo
    model_save_path = os.path.join(save_directory, "finbert_model")
    model.save_pretrained(model_save_path)

    # Guardar el tokenizador
    tokenizer_save_path = os.path.join(save_directory, "finbert_tokenizer")
    tokenizer.save_pretrained(tokenizer_save_path)

    print(f"Modelo guardado en {model_save_path}")
    print(f"Tokenizador guardado en {tokenizer_save_path}")


In [18]:


# Ejecución del flujo de trabajo
data_folder = "/content/drive/MyDrive/Bot 3.0/Historicos de cotizacion/Datos normalizados"
crypto_list = [
    'BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'EOS', 'BNB',
    'XLM', 'TRX', 'ADA', 'XTZ', 'XMR', 'DASH', 'ZEC',
    'ETC', 'NEO', 'MKR', 'XEM', 'DOGE', 'VET'
]

preprocessed_data = load_and_preprocess_data(crypto_list, data_folder)
finbert_model = fine_tune_finbert(preprocessed_data)
train_and_evaluate(finbert_model, preprocessed_data['train'], preprocessed_data['validation'])
save_model(finbert_model, '/content/drive/MyDrive/Bot 3.0/Inteligencia artificial/Compra Venta desde picos/Transformer')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: ignored

In [ ]:
# Aplicar la nueva función de detección y etiquetado de picos
preprocessed_data = unify_detect_label_peaks(preprocessed_data)

# Verificar las etiquetas
label_counts = preprocessed_data['Numeric_Signal'].value_counts()
print("Distribución de las etiquetas numéricas:")
print(label_counts)

# Verificar que solo haya dos clases únicas
unique_labels = preprocessed_data['Numeric_Signal'].unique()
print("Etiquetas únicas en los datos:", unique_labels)


In [ ]:
def check_labels(preprocessed_data):
    # Verificar si las señales numéricas son binarias (-1, 1)
    if preprocessed_data['Numeric_Signal'].isin([-1, 1]).all():
        print("Todas las señales numéricas son binarias (-1 para venta, 1 para compra).")
    else:
        print("Hay señales numéricas que no son binarias.")

    # Contar las señales de compra y venta
    signal_counts = preprocessed_data['Numeric_Signal'].value_counts()
    print(f"Señales de compra (1): {signal_counts.get(1, 0)}")
    print(f"Señales de venta (-1): {signal_counts.get(-1, 0)}")

    return signal_counts

# Usamos la función en nuestros datos preprocesados
signal_counts = check_labels(preprocessed_data)
print(signal_counts)
